### MySQL에서 video id를 가져온 후 video Info get

In [1]:
import numpy as np
import MySQLdb
import pandas as pd
import re
import requests
import schedule
import time

from datetime import datetime
from sqlalchemy import create_engine
from tqdm import tqdm

#### MySql

In [2]:
# connect mysql & Database 
conn = MySQLdb.connect(host="localhost", 
                       port=3306, 
                       user='root', 
                       password='******',
                       database='singAgain_db',
                       charset='utf8')

cursor = conn.cursor()

In [4]:
# create table
#sql = """CREATE TABLE video_info (currentTime DATETIME, videoId VARCHAR(255), publishTime DATETIME, 
#        videoTitle TINYTEXT, viewCount INT, likeCount INT, favoriteCount INT, commentCount INT)"""
#cursor.execute(sql)
#conn.commit()
#cursor = conn.cursor()

### Get video Info by video ID

In [22]:
# Get video ID from DB
sql = "Select * from video_id_list"
cursor.execute(sql)
video_id_list = cursor.fetchall()

In [23]:
# api id
my_id = '******************************'

In [24]:
def get_video_info(id_list):
    info_list = []
    for one_id in id_list:
        # 비디오 정보
        url = f'https://www.googleapis.com/youtube/v3/videos?part=id,snippet,statistics&id={one_id[0]}&key={my_id}'

        result = requests.get(url)
        result_json = result.json()

        if len(result_json['items']) == 0:
            continue

        one_info = []
        # 현재 시간
        c_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # 업로드 시간
        video_ptime = result_json['items'][0]['snippet']['publishedAt']
        video_ptime = video_ptime.replace("T", " ").replace("Z", "")
        # 제목
        title_value = result_json['items'][0]['snippet']['title']
        video_title = re.sub("[^ㄱ-힣1-9 ]+", "", title_value)
        # 정보
        video_info = list(result_json['items'][0]['statistics'].values())

        one_info.extend([c_time, one_id[0], video_ptime, video_title])
        one_info.extend(video_info)
        
        info_list.append(one_info)
        #print(info_list)
    return info_list

In [25]:
# get video id at mysql
def save_video_info(video_id_list):
    video_info_list = get_video_info(video_id_list)
    sql = f"""INSERT INTO video_info ()VALUES
                    (%s, %s, %s, %s, %s, %s, %s, %s)"""
    cursor.executemany(sql, video_info_list)
    conn.commit()

In [26]:
save_video_info(video_id_list)